# PoC: WeCookio multiagent Strands Agents

This notebook demonstrates the WeCookio CrewAI implementation with cost-optimized model selection using uv package manager.

### Collaborative Multi-agent with Graph

### Installing Required Packages with uv

In [1]:
# Install required packages using uv
!uv add ipykernel strands-agents==1.2.0 strands-agents-tools boto3 botocore pyyaml ipywidgets prompt-template

Resolved 224 packages in 0.78ms
Audited 213 packages in 0.08ms


### Import Libraries and Load Configuration

In [2]:
import yaml
import logging
from strands import Agent
from strands.multiagent import Swarm, GraphBuilder, SwarmResult

print("Libraries imported successfully!")

# Enable debug logs and print them to stderr
logging.getLogger("strands.multiagent").setLevel(logging.DEBUG)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)


Libraries imported successfully!


### Loading AWS Enviroment variables

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION=os.getenv("AWS_REGION")

### Load Configuration Files

In [4]:
#Check base folder for relative paths
import os
print (os.getcwd())
prefix = "strands/latest/"
# Load the integration configuration
with open(prefix+'config/integration_config.yaml', 'r') as file:
    integration_config = yaml.safe_load(file)
# Load the crew configuration
with open(prefix+'config/config.yaml', 'r') as file:
    strand_config = yaml.safe_load(file)
with open(prefix+'config/agents-swarm.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)
with open(prefix+'config/tasks-swarm.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("Configuration files loaded successfully!")
print(f"Integration config keys: {list(integration_config.keys())}")
print(f"Crew Orchestration keys: {list(strand_config.keys())}")
print(f"Agents config keys: {list(agents_config.keys())}")
print(f"Tasks config keys: {list(tasks_config.keys())}")

/home/e2its/dev/weCookio/researches
Configuration files loaded successfully!
Integration config keys: ['aws_bedrock', 'database', 'api_endpoints']
Crew Orchestration keys: ['crew', 'agents', 'tasks', 'integration_config', 'model_assignment', 'execution', 'memory', 'input_schema', 'output_schema', 'cost_optimization', 'performance_metrics', 'quality_standards', 'monitoring', 'example_input', 'example_output']
Agents config keys: ['Culinary Coordinator', 'Dietary Compliance Specialist', 'Ingredient Substitution Expert', 'Culinary Experience Optimizer', 'Quality Assurance Chef']
Tasks config keys: ['Enhance Recipe']


###  Model Selection based on complexity

In [5]:
def select_model_for_complexity(complexity, integration_config):
    """Select the appropriate model based on task complexity"""
    try:
        complexity = complexity + "_model"
        return integration_config["aws_bedrock"][complexity]
    except KeyError:
        raise ValueError(f"Invalid complexity level: {complexity}")

# Example usage
high_complexity_model = select_model_for_complexity("high_complexity", integration_config)
medium_complexity_model = select_model_for_complexity("medium_complexity", integration_config)
low_complexity_model = select_model_for_complexity("low_complexity", integration_config)
ultra_low_complexity_model = select_model_for_complexity("ultra_low_complexity", integration_config)

print(f"High complexity model: {high_complexity_model['model_id']}")
print(f"Medium complexity model: {medium_complexity_model['model_id']}")
print(f"Low complexity model: {low_complexity_model['model_id']}")
print(f"Ultra-Low complexity model: {ultra_low_complexity_model['model_id']}")

High complexity model: bedrock/amazon.nova-premier-v1:0
Medium complexity model: bedrock/amazon.nova-pro-v1:0
Low complexity model: bedrock/amazon.nova-lite-v1:0
Ultra-Low complexity model: bedrock/amazon.nova-micro-v1:0


### Create Agents with Model-Specific Configuration

In [6]:

from datetime import datetime
import boto3
from strands.models import BedrockModel
from prompt_template import PromptTemplate, InvalidTemplateKeysError, MissingTemplateValuesError, TemplateSerializationError

# Global metrics storage
execution_metrics = {
    'start_time': None,
    'total_tokens': 0,
    'total_cost': 0.0,
    'cycle_durations': [],
    'tool_usage': {},
    'errors': [],
    'agent_performance': {}
}

def safe_get_metrics(kwargs):
    """Safely extract metrics from kwargs"""
    try:
        metrics = kwargs.get('metrics')
        if metrics is None:
            return {
                'accumulated_usage': {'totalTokens': 0},
                'cycle_durations': [],
                'tool_metrics': {}
            }
        return metrics
    except Exception as e:
        print(f"⚠️  Warning: Could not extract metrics: {e}")
        return {
            'accumulated_usage': {'totalTokens': 0},
            'cycle_durations': [],
            'tool_metrics': {}
        }

def event_loop_tracker(**kwargs):
    """Enhanced event loop tracker with proper error handling and metrics collection"""
    try:
        # Track event loop lifecycle
        if kwargs.get("init_event_loop", False):
            print("🔄 Event loop initialized")
            execution_metrics['start_time'] = datetime.now()
        elif kwargs.get("start_event_loop", False):
            print("▶️ Event loop cycle starting")
        elif kwargs.get("start", False):
            print("📝 New cycle started")
        elif "message" in kwargs:
            #print(f"📬 New message created: {kwargs['message']['role']}")
            message=kwargs['message']['content'][0]
            if message.get("text"):
                print(f"📬 New message created: {message['text']}")
            else:
                print(f'''📬 New Tool Call created {message['toolResult']['status']}:{message['toolResult']['content'][0]['text']}''')
        elif kwargs.get("complete", False):
            print("✅ Cycle completed")
            # Update global metrics
            metrics = safe_get_metrics(kwargs)
            if metrics['cycle_durations']:
                execution_metrics['cycle_durations'].extend(metrics['cycle_durations'])
            if metrics['accumulated_usage']['totalTokens']:
                execution_metrics['total_tokens'] += metrics['accumulated_usage']['totalTokens']
        elif kwargs.get("force_stop", False):
            print(f"🛑 Event loop force-stopped: {kwargs.get('force_stop_reason', 'unknown reason')}")
            execution_metrics['errors'].append({
                'type': 'force_stop',
                'reason': kwargs.get('force_stop_reason', 'unknown'),
                'timestamp': datetime.now().isoformat()
            })

        # Track tool usage
        if "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
            tool_name = kwargs["current_tool_use"]["name"]
            print(f"🔧 Using tool: {tool_name}")
            execution_metrics['tool_usage'][tool_name] = execution_metrics['tool_usage'].get(tool_name, 0) + 1

        # Show metrics when data is available
        #if "data" in kwargs:
            # Only show first 20 chars of each chunk for demo purposes
            #data_snippet = kwargs["data"][:20] + ("..." if len(kwargs["data"]) > 20 else "")
            #print(f"📟 Text: {data_snippet}")   
    except Exception as e:
        error_msg = f"Error in event loop tracker: {e}"
        print(f"❌ {error_msg}")
        execution_metrics['errors'].append({
            'type': 'event_loop_tracker_error',
            'message': str(e),
            'timestamp': datetime.now().isoformat()
        })
    
def agent_description_composer(agent_config):
    """Compose the agent description"""
    return f"""
    Role: {agent_config.get("role", None)}
    Goal: {agent_config.get("goal", None)}
    Backstory: {agent_config.get("backstory", None)}
    """

def create_agent_with_model(agent_config, model_config, **kwargs):
    """Create an agent with specific model configuration"""
    llm = BedrockModel(
        model_id=model_config["model_id"].split("/")[-1],
        temperature=model_config.get("model_kwargs").get("temperature", 0.6),
        top_p=model_config.get("model_kwargs").get("top_p", 0.9),
        max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000)
    )

    args = {}
    args["model"]=llm
    args["system_prompt"]=agent_config.get("prompt", None)
    args["name"]=agent_config.get("name", None)
    args["description"]=agent_description_composer(agent_config)
    args["callback_handler"]=event_loop_tracker
    
    if not agent_config.get("verbose",True):
        args["callback_handler"]=None
    
    return Agent(**args)

def print_agent_info(agent, model_config) -> None:
    """Print agent creation information."""
    print(
        f"Created agent: {agent.name}\n"
        f"Agent Description: {agent.description}\n"
        f"Using model: {agent.model.get_config()}"
    )
    print("\n")

In [7]:

#agent_name = "Culinary Coordinator"
agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_coordinator_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_coordinator_agent, model_config)

agent_name = "Dietary Compliance Specialist"
agent_config = agents_config["Dietary Compliance Specialist"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
dietary_compliance_specialist_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(dietary_compliance_specialist_agent, model_config)

agent_name = "Ingredient Substitution Expert"
agent_config = agents_config["Ingredient Substitution Expert"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
ingredient_substitution_expert_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(ingredient_substitution_expert_agent, model_config)

agent_name = "Culinary Experience Optimizer"
agent_config = agents_config["Culinary Experience Optimizer"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_experience_optimizer_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_experience_optimizer_agent, model_config)

agent_name = "Quality Assurance Chef"
agent_config = agents_config["Quality Assurance Chef"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(strand_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
quality_assurance_chef_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(quality_assurance_chef_agent, model_config)


Created agent: Culinary Coordinator
Agent Description: 
    Role: Master Chef & Project Manager
    Goal: Orchestrate recipe adaptation ensuring maximum culinary excellence and strict dietary compliance
    Backstory: Executive chef with 20+ years experience in dietary adaptations and team coordination
    
Using model: {'model_id': 'amazon.nova-lite-v1:0', 'temperature': 0.4, 'top_p': 0.9, 'max_tokens': 2000}


Created agent: Dietary Compliance Specialist
Agent Description: 
    Role: Nutritionist & Food Safety Expert
    Goal: Ensure 100% compliance with dietary restrictions and food safety standards
    Backstory: Certified nutritionist and food safety expert with deep knowledge of allergens and dietary requirements
    
Using model: {'model_id': 'amazon.nova-micro-v1:0', 'temperature': 0.2, 'top_p': 0.9, 'max_tokens': 1000}


Created agent: Ingredient Substitution Expert
Agent Description: 
    Role: Culinary and dietary sustitutions Specialist
    Goal: Find optimal ingredient sub

### Create a Flow

In [8]:
# Example recipe for analysis
recipe_request = {
    "food_name": "Canelones de ternera XXL al Pedro Ximenez",
    "servings": 4,
    "intolerances": [
        "Milk",
        "Eggs",
        "Beef",
        "Corn",
        "Gluten",
        "Rice",
        "Garlic"
    ],
    "exclusions": ["Pepper"],
    "preferences": [],
    "output_language": "spanish",
    "country": "Spain"
}


In [9]:
## Support functions

def variable_injection(task:dict, recipe_context:dict):
    try:
        prompt = PromptTemplate(name=task.get("name"), template=f'''{task.get("description")}''').to_string(**recipe_context)
    except MissingTemplateValuesError as e:
        print(f"Missing values: {e.missing_values}")  # {'name'}
    except InvalidTemplateKeysError as e:
        print(f"Invalid keys: {e.invalid_keys}")  # {'name'}
    except TemplateSerializationError as e:
        print(f"Serialization error: {e.message}")  # 'Invalid template string'
    except Exception as e:
        print(f"Unexpected error: {e}")  # 'Unexpected error'
    return prompt
    

In [10]:
tasks = {
    "enhance_recipe": {
        "description": f'''{variable_injection(tasks_config['Enhance Recipe'], recipe_request)}''',
        "dependencies": []
    }
}
print(tasks['enhance_recipe'].get("description"))

Enhance the recipe with the following details:
## RECIPE CONTEXT
- **Recipe Name**: Canelones de ternera XXL al Pedro Ximenez
- **Servings**: 4
- **Dietary Restrictions**: ["Milk", "Eggs", "Beef", "Corn", "Gluten", "Rice", "Garlic"], ["Pepper"]
- **Preferences**: []
- **Target Country**: Spain
- **Output Language**: spanish

## TASK OBJECTIVE
Transform the requested recipe to be 100% compliant with dietary restrictions and adapted to preferences 
while maximizing culinary experience and preserving authentic flavor/texture characteristics and provide the expected output indicated bellow.

**Expected output:**
- Normalize the recipe and instructions to the target Spain
- Return the recipe and instructions in the target Spain format and language spanish
- Validation status for Quality checks


In [11]:
# Create a swarm with agents
substitution_swarm = Swarm(
    [ingredient_substitution_expert_agent, dietary_compliance_specialist_agent],
    max_handoffs=15,
    max_iterations=5,
    execution_timeout=90.0,  # 1.5 minutes
    node_timeout=30.0,       # 30 seconds per agent
    repetitive_handoff_detection_window=5,  # There must be >= 3 unique agents in the last 8 handoffs
    repetitive_handoff_min_unique_agents=1    
)

DEBUG | strands.multiagent.swarm | nodes=<['Ingredient Substitution Expert', 'Dietary Compliance Specialist']> | initialized swarm with nodes
DEBUG | strands.multiagent.swarm | tool_count=<1>, node_count=<2> | injected coordination tools into agents


### Run the Flow

In [12]:
graph = GraphBuilder()

graph.add_node(culinary_coordinator_agent, "culinary_coordinator_agent")
graph.add_node(substitution_swarm, "ingredient_substitution_swarm")
graph.add_node(culinary_experience_optimizer_agent, "culinary_experience_optimizer_agent")
graph.add_node(quality_assurance_chef_agent, "quality_assurance_chef_agent")

graph.add_edge("culinary_coordinator_agent", "ingredient_substitution_swarm")
graph.add_edge("culinary_coordinator_agent", "quality_assurance_chef_agent")
graph.add_edge("culinary_coordinator_agent", "culinary_experience_optimizer_agent")


graph.set_entry_point("culinary_coordinator_agent")

In [15]:
try:
    # Run the Graph
    result = graph.build()(tasks['enhance_recipe'].get("description"))
    # Access the final result
    # Access the results
    print(f"\nStatus: {result.status}")
    print(f"Execution order: {[node.node_id for node in result.execution_order]}")  
except Exception as e:
    print(f"Error running Workflow: {e}")
    print("Make sure you have AWS credentials configured and the required permissions.")


DEBUG | strands.multiagent.graph | task=<Enhance the recipe with the following details:
## RECIPE CONTEXT
- **Recipe Name**: Canelones de ternera XXL al Pedro Ximenez
- **Servings**: 4
- **Dietary Restrictions**: ["Milk", "Eggs", "Beef", "Corn", "Gluten", "Rice", "Garlic"], ["Pepper"]
- **Preferences**: []
- **Target Country**: Spain
- **Output Language**: spanish

## TASK OBJECTIVE
Transform the requested recipe to be 100% compliant with dietary restrictions and adapted to preferences 
while maximizing culinary experience and preserving authentic flavor/texture characteristics and provide the expected output indicated bellow.

**Expected output:**
- Normalize the recipe and instructions to the target Spain
- Return the recipe and instructions in the target Spain format and language spanish
- Validation status for Quality checks> | starting graph execution
DEBUG | strands.multiagent.graph | node_id=<culinary_coordinator_agent> | executing node


🔄 Event loop initialized
📝 New cycle started
▶️ Event loop cycle starting


DEBUG | strands.multiagent.graph | node_id=<culinary_coordinator_agent>, execution_time=<3517ms> | node completed successfully
DEBUG | strands.multiagent.graph | from=<culinary_coordinator_agent>, to=<ingredient_substitution_swarm> | edge ready via satisfied condition
DEBUG | strands.multiagent.graph | from=<culinary_coordinator_agent>, to=<culinary_experience_optimizer_agent> | edge ready via satisfied condition
DEBUG | strands.multiagent.graph | from=<culinary_coordinator_agent>, to=<quality_assurance_chef_agent> | edge ready via satisfied condition
DEBUG | strands.multiagent.graph | node_id=<ingredient_substitution_swarm> | executing node
DEBUG | strands.multiagent.swarm | starting swarm execution
DEBUG | strands.multiagent.swarm | current_node=<Ingredient Substitution Expert> | starting swarm execution with node
DEBUG | strands.multiagent.swarm | max_handoffs=<15>, max_iterations=<5>, timeout=<90.0>s | swarm execution config
DEBUG | strands.multiagent.swarm | current_node=<Ingredie

📬 New message created: **Receta Mejorada: Canelones XXL de ternera al Pedro Ximénez**

**Ingredientes (para 4 personas):**
- **Pollo**: 500g de pechuga de pollo magra (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Aceite de oliva**: 2 cucharadas
- **Cebolla**: 1 cebolla mediana (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Zanahoria**: 1 zanahoria mediana (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Apio**: 2 tallos de apio (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Vino Pedro Ximénez**: 100 ml
- **Consomé de pollo**: 500 ml (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Leche de coco**: 100 ml (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Pasta para canelones**: 12 hojas de pasta para canelones sin gluten
- **Queso rallado**: 100g de queso rallado sin lácteos y sin

DEBUG | strands.multiagent.graph | node_id=<culinary_experience_optimizer_agent>, execution_time=<3031ms> | node completed successfully
DEBUG | strands.multiagent.graph | node_id=<quality_assurance_chef_agent>, execution_time=<3220ms> | node completed successfully


...

📬 New message created: <thinking> The task has been handed off to the Ingredient Substitution Expert for confirmation. I will now wait for their confirmation regarding the compliance of the recipe with dietary restrictions and the maintenance of original flavor and texture. </thinking> 
📬 New Tool Call created success:Handed off to Ingredient Substitution Expert: Please confirm that the recipe complies with all dietary restrictions and that the substitutions maintain the original flavor and texture as much as possible.
📝 New cycle started
📝 New cycle started
▶️ Event loop cycle starting
🔧 Using tool: handoff_to_agent


DEBUG | strands.multiagent.swarm | from_node=<Ingredient Substitution Expert>, to_node=<Ingredient Substitution Expert> | handed off from agent to agent


📬 New message created: <thinking> The task has been handed off to the Ingredient Substitution Expert for confirmation. I will now wait for their confirmation regarding the compliance of the recipe with dietary restrictions and the maintenance of original flavor and texture. </thinking> 
📬 New Tool Call created success:Handed off to Ingredient Substitution Expert: Please confirm that the recipe complies with all dietary restrictions and that the substitutions maintain the original flavor and texture as much as possible.
📝 New cycle started
📝 New cycle started
▶️ Event loop cycle starting


ERROR | strands.multiagent.swarm | node=<Dietary Compliance Specialist>, timeout=<30.0>s | node execution timed out after timeout
Traceback (most recent call last):
  File "/home/e2its/.local/share/uv/python/cpython-3.13.5-linux-x86_64-gnu/lib/python3.13/asyncio/tasks.py", line 507, in wait_for
    return await fut
           ^^^^^^^^^
  File "/home/e2its/dev/weCookio/.venv/lib/python3.13/site-packages/strands/multiagent/swarm.py", line 590, in _execute_node
    result = await node.executor.invoke_async(node_input)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/e2its/dev/weCookio/.venv/lib/python3.13/site-packages/strands/agent/agent.py", line 398, in invoke_async
    async for event in events:
        _ = event
  File "/home/e2its/dev/weCookio/.venv/lib/python3.13/site-packages/strands/agent/agent.py", line 508, in stream_async
    async for event in events:
    ...<2 lines>...
            yield event["callback"]
  File "/home/e2its/dev/weCookio/.venv/lib/pyth

🔧 Using tool: handoff_to_agent

Status: Status.COMPLETED
Execution order: ['culinary_coordinator_agent', 'culinary_experience_optimizer_agent', 'quality_assurance_chef_agent', 'ingredient_substitution_swarm']


In [16]:
# Ver resultado del agente coordinador
agent_name = 'culinary_coordinator_agent'
agent_result=result.results[agent_name]

print(f"\n=== RESULTADO DE {agent_name} ===")
print(agent_result.result.message['content'][0]['text'])
print(f"Status: {agent_result.status}")
print(f"Execution time: {agent_result.execution_time}ms")


=== RESULTADO DE culinary_coordinator_agent ===
**Receta Mejorada: Canelones XXL de ternera al Pedro Ximénez**

**Ingredientes (para 4 personas):**
- **Pollo**: 500g de pechuga de pollo magra (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Aceite de oliva**: 2 cucharadas
- **Cebolla**: 1 cebolla mediana (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Zanahoria**: 1 zanahoria mediana (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Apio**: 2 tallos de apio (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Vino Pedro Ximénez**: 100 ml
- **Consomé de pollo**: 500 ml (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Leche de coco**: 100 ml (sin gluten, sin lácteos, sin huevo, sin maíz, sin gluten, sin arroz, sin ajo)
- **Pasta para canelones**: 12 hojas de pasta para canelones sin gluten
- **Queso rallado**: 100g de queso